In [ ]:
# aid.ipynb
# Author: Kyle Larson
# Purpose top secret
import numpy as np 
import soundfile as sf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
# Stack overflow was consulted in order to parse '.flac' 
#path = '../input/rfcx-species-audio-detection/train/00204008d.flac'
#data, samplerate = sf.read(path) #this method is thanks to Harry Moreno 
#from his Stack Overflow response on June 13 '18 at the following URL:
#https://stackoverflow.com/questions/50804170/load-flac-file-in-python-same-as-scipy-or-librosa


In [ ]:
import os
file_name_list = os.listdir('../input/rfcx-species-audio-detection/train')
#the following sequential open strategy was found not to work due to frequency mismatch between id_data and filenames
for i in range(len(file_name_list)):#len(id_data) #10 files ~ 5 seconds w/ RAM ~25%
    data, samplerate = sf.read('../input/rfcx-species-audio-detection/train/' +file_name_list[i])
    new_df1[file_name_list[i]] = np.array(data)
    if(i==10):
        break
new_df1.head() 

In [ ]:
#at 1 min 5 sec the following error occured:
#Your notebook tried to allocate more memory than is available. It has restarted.
#thats 65 seconds or 130 files that can occupy ram at once